In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-06-28 10:57:27 __main__ INFO     torch.__version__='2.7.1+cu126', torch.version.cuda='12.6'
2025-06-28 10:57:28 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-06-28 10:57:28 __main__ INFO     transformers.__version__='4.53.0'


In [3]:
from src.utils.training_utils import get_device_map

model_key = "meta-llama/Llama-3.3-70B-Instruct"

device_map = get_device_map(model_key, 32, n_gpus=8)
device_map

2025-06-28 10:57:40 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/gio/mechinterp, stdin=None, shell=False, universal_newlines=False)
2025-06-28 10:57:40 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/gio/mechinterp, stdin=None, shell=False, universal_newlines=False)
x: 42
y: hello


{'model.embed_tokens': 7,
 'model.norm': 7,
 'model.rotary_emb': 7,
 'lm_head': 7,
 'model.layers.0': 0,
 'model.layers.1': 1,
 'model.layers.2': 2,
 'model.layers.3': 3,
 'model.layers.4': 4,
 'model.layers.5': 5,
 'model.layers.6': 6,
 'model.layers.7': 7,
 'model.layers.8': 0,
 'model.layers.9': 1,
 'model.layers.10': 2,
 'model.layers.11': 3,
 'model.layers.12': 4,
 'model.layers.13': 5,
 'model.layers.14': 6,
 'model.layers.15': 7,
 'model.layers.16': 0,
 'model.layers.17': 1,
 'model.layers.18': 2,
 'model.layers.19': 3,
 'model.layers.20': 4,
 'model.layers.21': 5,
 'model.layers.22': 6,
 'model.layers.23': 7,
 'model.layers.24': 0,
 'model.layers.25': 1,
 'model.layers.26': 2,
 'model.layers.27': 3,
 'model.layers.28': 4,
 'model.layers.29': 5,
 'model.layers.30': 6,
 'model.layers.31': 7,
 'model.layers.32': 0,
 'model.layers.33': 1,
 'model.layers.34': 2,
 'model.layers.35': 3,
 'model.layers.36': 4,
 'model.layers.37': 5,
 'model.layers.38': 6,
 'model.layers.39': 7,
 'model

In [4]:
os.getcwd()

'/disk/u/gio/mechinterp'

In [5]:
from src.models import ModelandTokenizer

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    device_map=device_map
)

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-06-28 10:58:45 src.models INFO     loaded model <models/meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:7


In [6]:
from src.functional import get_hs

class Patchscope:
    def __init__(self, mt):
        self.model = mt.model
        self.tokenizer = mt.tokenizer

    def run(
        self, 
        source_prompt,
        target_prompt,
        source_layer_idx,
        target_layer_idx,
        target_token_idx
    ):
        # Feed source prompt to model and capture hidden state
        source_inputs = self.tokenizer(source_prompt, return_tensors="pt")

        # Register a forward hook on the specified layer
        source_hook_handle = self.model.model.layers[source_layer_idx].register_forward_hook(get_hidden_state_hook)

        with torch.no_grad()

NameError: name 'cla' is not defined